In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageTk
import numpy as np
import joblib
import os
from catboost import CatBoostRegressor

# Initialize the main window
root = tk.Tk()
root.title("Hydraulic Jump Roller Length Prediction Model")
root.geometry("1024x700")  # Set a fixed window size
root.configure(bg="#f0f0f5")

# Title
title_label = tk.Label(root, text="Hydraulic Jump Roller Length Prediction Model",
                       font=("Palatino Linotype", 20, "bold"), fg="#000000", bg="#f0f0f5")
title_label.pack(pady=(10, 5))

# Developers' Information
dev_label = tk.Label(root, text="Developed by: Eng. Mohamed Kamel Elshaarawy & Abdelrahman Kamal Hamed",
                     font=("CMU Concrete", 14, "bold"), fg="#000000", bg="#f0f0f5")
dev_label.pack()

# Affiliation
affiliation_label = tk.Label(root, text="Civil Engineering Department, Faculty of Engineering, Horus University-Egypt, New Damietta 34517, Egypt",
                             font=("CMU Concrete", 12), fg="#0000FF", bg="#f0f0f5")
affiliation_label.pack(pady=(0, 10))

# Create a canvas for the image
canvas = tk.Canvas(root, width=550, height=220, bg="#FFFFFF")
canvas.pack(pady=10)

# Load and scale the image 
image_path = "C:/Users/asus1/Desktop/image2.png"
try:
    image = Image.open(image_path)
    image = image.resize((int(image.width * 1.25), int(image.height * 1.25)), Image.LANCZOS)
    root.image = ImageTk.PhotoImage(image)
    canvas.create_image(270, 100, image=root.image, anchor="center")
except Exception as e:
    messagebox.showerror("Image Error", f"Unable to load image: {e}. The application will continue without the image.")

# Function to create labels within frames
def create_label(parent, text, font, fg, bg, x, y):
    label = tk.Label(parent, text=text, font=font, fg=fg, bg=bg)
    label.place(x=x, y=y)
    return label

# Load the CatBoost model
model_path = os.path.expanduser("C:/Users/asus1/Desktop/CTB.joblib")
try:
    model = joblib.load(model_path)
except Exception as e:
    messagebox.showerror("Model Error", f"Unable to load model: {e}")
    root.destroy()
    exit()

# Create a frame for input parameters
input_frame = ttk.LabelFrame(root, text="Input Parameters", padding=(10, 10))
input_frame.pack(pady=15, padx=10, fill="x")
param_labels = [
    'Upstream Froude Number (Fr₁)',
    'Relative HJ depths (h₂/h₁)',
    'Relative bed roughness to HJ Initial Depth (ks/h₁)'
]
entry_fields = ['Fr₁', 'h₂/h₁', 'ks/h₁']
entries = {}

# Position Input Labels and Fields
for i, (text, field) in enumerate(zip(param_labels, entry_fields), start=1):
    label = tk.Label(input_frame, text=text, font=("Georgia", 14), fg="#00008B", bg="#FFFFFF")
    label.grid(row=i, column=0, padx=10, pady=5, sticky="w")
    entry = tk.Entry(input_frame, font=("Georgia", 14, "bold"))
    entry.grid(row=i, column=1, padx=10, pady=5, sticky="w")
    entries[field] = entry

# Create a frame for output result
output_frame = ttk.LabelFrame(root, text="Prediction Result", padding=(5, 10))
output_frame.pack(pady=10, padx=10, fill="x")
label_result = tk.Label(output_frame, text="Predicted Relative Roller Length (Lr/h₁): ", font=("Georgia", 16), fg="#FF0000", bg="#FFFFFF")
label_result.pack(padx=10, pady=10)

# Predict Button Command
def predict():
    try:
        fr1 = float(entries['Fr₁'].get())
        h2_h1 = float(entries['h₂/h₁'].get())
        ks_h1 = float(entries['ks/h₁'].get())

        if any(val <= 0 for val in [fr1, h2_h1, ks_h1]):
            raise ValueError("All inputs must be positive.")

        prediction = model.predict([[fr1, h2_h1, ks_h1]])[0]
        label_result.config(text=f'Predicted Relative Roller Length (Lr/h₁): {prediction:.4f}')
    except ValueError:
        messagebox.showerror("Input Error", "Please enter valid, positive numbers for all inputs.")
    except Exception as e:
        messagebox.showerror("Prediction Error", f"An error occurred during prediction: {e}")

# Add frame for buttons
buttons_frame = tk.Frame(root, bg='#f0f0f5')
buttons_frame.pack(pady=10)
buttons_frame.place(x=400, y=570)

predict_button = ttk.Button(buttons_frame, text='Predict', command=predict, width=15)
predict_button.grid(row=0, column=0, padx=10)

clear_button = ttk.Button(buttons_frame, text='Clear', command=lambda: [entry.delete(0, tk.END) for entry in entries.values()], width=15)
clear_button.grid(row=0, column=1, padx=10)


# Run the main loop
root.mainloop()
